In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install stable-baselines3[extra]

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu113/torch-1.10.1%2Bcu113-cp39-cp39-win_amd64.whl (2442.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 1.6.2 requires torch>=1.11, but you have torch 1.10.1+cu113 which is incompatible.


  Using cached torch-1.13.0-cp39-cp39-win_amd64.whl (167.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1+cu113
    Uninstalling torch-1.10.1+cu113:
      Successfully uninstalled torch-1.10.1+cu113


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.2+cu113 requires torch==1.10.1, but you have torch 1.13.0 which is incompatible.
torchaudio 0.10.1+cu113 requires torch==1.10.1, but you have torch 1.13.0 which is incompatible.


In [2]:
# Import the game
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
# Setup the game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [4]:
# Import other needed libabries
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
import matplotlib.pyplot as plt

In [5]:
# Create the environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [6]:
state = env.reset()

In [7]:
state, reward, done, info = env.step([5])

In [8]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [9]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [10]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [11]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [12]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

Using cpu device
Wrapping the env in a VecTransposeImage.


In [13]:
# Training the AI model
model.learn(total_timesteps=20000, callback=callback)

Logging to ./logs/PPO_1


C:\Users\Ryevencarm\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 34  |
|    iterations      | 1   |
|    time_elapsed    | 14  |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 2             |
|    time_elapsed         | 108           |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 6.2959734e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0026        |
|    learning_rate        | 1e-06         |
|    loss                 | 216           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000131     |
|    value_loss           | 512           |
-------------------------------------------
-----

-----------------------------------------
| time/                   |             |
|    fps                  | 6           |
|    iterations           | 13          |
|    time_elapsed         | 1028        |
|    total_timesteps      | 6656        |
| train/                  |             |
|    approx_kl            | 6.19411e-06 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.95       |
|    explained_variance   | -0.00139    |
|    learning_rate        | 1e-06       |
|    loss                 | 0.139       |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.000171   |
|    value_loss           | 0.268       |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 14           |
|    time_elapsed         | 1119         |
|    total_timesteps      | 7

-------------------------------------------
| time/                   |               |
|    fps                  | 6             |
|    iterations           | 24            |
|    time_elapsed         | 1978          |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 1.9255094e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0184        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.109         |
|    n_updates            | 230           |
|    policy_gradient_loss | -0.000274     |
|    value_loss           | 0.474         |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 25           |
|    time_elapsed         | 2060    

-------------------------------------------
| time/                   |               |
|    fps                  | 6             |
|    iterations           | 35            |
|    time_elapsed         | 2933          |
|    total_timesteps      | 17920         |
| train/                  |               |
|    approx_kl            | 0.00018513319 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | 0.00624       |
|    learning_rate        | 1e-06         |
|    loss                 | 206           |
|    n_updates            | 340           |
|    policy_gradient_loss | 2.25e-05      |
|    value_loss           | 428           |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 6             |
|    iterations           | 36            |
|    time_elapsed         | 3014

In [14]:
model.save('testmodel')

In [15]:
# Load model
model = PPO.load('./train/best_model_20000')

In [16]:
state = env.reset()

In [17]:
pip install --upgrade gym==0.20.0


     ---------------------------------------- 1.6/1.6 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gym: filename=gym-0.20.0-py3-none-any.whl size=1650451 sha256=add8702837ccd01c56c230b59cdf77c1553b355370ca90f01de794500f758df8
  Stored in directory: c:\users\ryevencarm\appdata\local\pip\cache\wheels\9f\91\e5\ac160083c57e371f1137b11482894fca821597dc5961fb9e8c
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.21.0
    Uninstalling gym-0.21.0:
      Successfully uninstalled gym-0.21.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 1.6.2 requires gym==0.21, but you have gym 0.20.0 which is incompatible.


In [19]:
#Run the game 
import gym
from gym import wrappers

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
for _ in range(3600):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done: break
env.close()

In [20]:
import io
import base64
from IPython.display import HTML

video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))